### Consigna

- Elegir algún corpus de texto de su preferencia (conviene que sea algo de lo cual ustedes tengan algo de conocimiento al respecto para poder después interpretar la similaridades de embeddings que estudien). Puede ser literatura, texto técnico, blogs, artículos, etc...
- Entrenar embeddings con el  corpus elegido utilizando Gensim. Pueden intentar modificar varios parámetros: tipo de modelo (cbow/skipgram), tamaño de vocabulario, tamaño de ventana de contexto, negative samples (que trataremos de hablar algo porque no hubo tiempo la vez pasada).
- Con alguna de versión de embeddigns que hayan entrenado (o con todas!) estudiar similaridades de términos que crean interesantes. Graficarlos en 2D y si quieren en 3D y destacar casos interesantes para su corpus. Interpretarlos del conocimiento a priori que tengan del corpus.


<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



# Consigna




### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2024-04-01 05:41:16--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2024-04-01 05:41:16--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2024-04-01 05:41:16--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['adele.txt',
 'janisjoplin.txt',
 'lil-wayne.txt',
 'joni-mitchell.txt',
 'nickelback.txt',
 'drake.txt',
 'dickinson.txt',
 'lin-manuel-miranda.txt',
 'Lil_Wayne.txt',
 'bob-dylan.txt',
 'kanye-west.txt',
 'prince.txt',
 'blink-182.txt',
 'bruno-mars.txt',
 'bieber.txt',
 'paul-simon.txt',
 'johnny-cash.txt',
 'bruce-springsteen.txt',
 'al-green.txt',
 'dj-khaled.txt',
 'cake.txt',
 'bjork.txt',
 'amy-winehouse.txt',
 'nursery_rhymes.txt',
 'jimi-hendrix.txt',
 'kanye.txt',
 'eminem.txt',
 'nirvana.txt',
 'lady-gaga.txt',
 'beatles.txt',
 'disney.txt',
 'britney-spears.txt',
 'alicia-keys.txt',
 'bob-marley.txt',
 'rihanna.txt',
 'radiohead.txt',
 'notorious-big.txt',
 'notorious_big.txt',
 'dr-seuss.txt',
 'patti-smith.txt',
 'Kanye_West.txt',
 'r-kelly.txt',
 'michael-jackson.txt',
 'leonard-cohen.txt',
 'dolly-parton.txt',
 'ludacris.txt',
 'lorde.txt',
 'missy-elliott.txt',
 'nicki-minaj.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65953.953125
Loss after epoch 2: 65934.265625
Loss after epoch 3: 65711.96875
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.46875
Loss after epoch 6: 64058.8125
Loss after epoch 7: 64755.4375
Loss after epoch 8: 62585.5625
Loss after epoch 9: 60415.25
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.1875
Loss after epoch 12: 57726.4375
Loss after epoch 13: 56493.125
Loss after epoch 14: 55842.5625
Loss after epoch 15: 55862.5
Loss after epoch 16: 51695.8125
Loss after epoch 17: 49815.5
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087147116661072),
 ('sleep', 0.8649678230285645),
 ('help', 0.8594305515289307),
 ('try', 0.8456019759178162),
 ('cry', 0.8387227654457092),
 ('little', 0.8273633718490601),
 ('not', 0.8181102871894836),
 ('seems', 0.8175203204154968),
 ('twist', 0.8154979348182678),
 ('peace', 0.812263011932373)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786975860595703),
 ('our', -0.24495810270309448),
 ('come', -0.2613270878791809),
 ('on', -0.26722386479377747),
 ('bang', -0.27329546213150024),
 ('five', -0.2741836607456207),
 ('six', -0.2774038314819336),
 ('baby', -0.2776353061199188),
 ('work', -0.2790372967720032),
 ('four', -0.2792743146419525)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786694049835205),
 ('three', 0.9761051535606384),
 ('six', 0.966891348361969),
 ('two', 0.9565720558166504),
 ('seven', 0.954232931137085),
 ('sixty', 0.9003373980522156),
 ('one', 0.816501259803772),
 ('us', 0.7762899398803711),
 ('crying', 0.772737979888916),
 ('strawberry', 0.7672452330589294)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409592747688293),
 ('buy', 0.9399508237838745),
 ('much', 0.8996850848197937),
 ('hide', 0.8465201258659363),
 ('just', 0.8415144085884094)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["die"])

[('come', -0.5779027938842773),
 ('together', -0.5920610427856445),
 ('sky', -0.593083918094635),
 ('yellow', -0.5938284397125244),
 ('on', -0.5992255210876465),
 ('day', -0.6002489328384399),
 ('our', -0.6037141680717468),
 ('la', -0.605088472366333),
 ('ah', -0.6060557961463928),
 ('back', -0.6076730489730835)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736822  0.21460655 -0.05408018  0.06707036  0.13059464 -0.25713933
  0.00711611  0.26743424 -0.03100746  0.259963    0.21179457 -0.08465387
 -0.26272097  0.308386   -0.07934437 -0.13357355  0.2994755   0.18062147
 -0.13111673 -0.11857472  0.03468326  0.1125607   0.08478875  0.08090881
 -0.17622267  0.03122034 -0.08725598  0.23746666 -0.11507358 -0.29119617
 -0.03650665 -0.3322312   0.23262788  0.10366543 -0.18365337  0.08299578
  0.14315338 -0.19761457 -0.00757894  0.1119568  -0.10421463 -0.13083619
 -0.03264419 -0.04687435  0.11330808  0.10231452 -0.16202034 -0.10347028
  0.03659901 -0.08524423 -0.34872082 -0.02154255  0.25938764  0.16540806
  0.0310861   0.09505633  0.24452917 -0.09859891  0.18298703  0.0645447
  0.016967   -0.3259168  -0.04774074 -0.05418535 -0.04338397 -0.09137302
  0.00889874  0.11636022 -0.25662988  0.05426472  0.08915784  0.04541444
  0.20835714 -0.19101828  0.32210803  0.19337991  0.09080223 -0.00362429
 -0.18664253 -0.07053486 -0.1213743  -0.03477469 -0.

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689683079719543),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385539054870605)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689682483673096),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385538458824158),
 ('hope', 0.8361446261405945)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

# Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

Vamos a importar las librerias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

Importamos el corpus

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Información del Corpus: https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews?resource=download

Usability
7.94

License
CC0: Public Domain

In [ ]:
# Descargar la carpeta de dataset
import os
import platform

!pwd
if os.access('./Reviews.csv', os.F_OK) is False or os.access('./hashes.txt', os.F_OK) is False or os.access('./database.sqlite', os.F_OK) is False:
  !unzip -q /content/drive/MyDrive/Maestria/NLP/Amazon_Fine_Food_Reviews.zip
else:
    print("El dataset ya se encuentra descargado")

/content


In [ ]:
# Posibles bandas
os.listdir("./")

['.config',
 'drive',
 'database.sqlite',
 'Reviews.csv',
 'hashes.txt',
 'sample_data']

In [ ]:
# prompt: read ./Reviews.csv and put it into a df

import pandas as pd
df = pd.read_csv('./Reviews.csv')

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Ahora vamos entrenar dos modelos, para las reviews de 1 estrella, es decir las negativas, y las de 5 estrellas, es decir las positivas. Luego veremos las diferencias.

## 1 estrella

Vamos a tomar solo las reviews negativas, es decir con 1 sola estrella

In [ ]:
#Me voy a quedar solo con la columna Text y las reviews que son apriori malas
t=df[df['Score'] == 1]['Text'].to_frame()

In [ ]:
t.head(30)

,Text
1,Product arrived labeled as Jumbo Salted Peanut...
12,My cats have been happily eating Felidae Plati...
26,"The candy is just red , No flavor . Just plan..."
50,"This oatmeal is not good. Its mushy, soft, I d..."
62,Arrived in 6 days and were so stale i could no...
73,Buyer Beware Please! This sweetener is not for...
75,No tea flavor at all. Just whole brunch of art...
99,I fed this to my Golden Retriever and he hated...
153,These singles sell for $2.50 - $3.36 at the st...
164,Seriously this product was as tasteless as the...


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for index, row in t.iterrows():
  sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['product',
  'arrived',
  'labeled',
  'as',
  'jumbo',
  'salted',
  'peanuts',
  'the',
  'peanuts',
  'were',
  'actually',
  'small',
  'sized',
  'unsalted',
  'not',
  'sure',
  'if',
  'this',
  'was',
  'an',
  'error',
  'or',
  'if',
  'the',
  'vendor',
  'intended',
  'to',
  'represent',
  'the',
  'product',
  'as',
  'jumbo'],
 ['my',
  'cats',
  'have',
  'been',
  'happily',
  'eating',
  'felidae',
  'platinum',
  'for',
  'more',
  'than',
  'two',
  'years',
  'i',
  'just',
  'got',
  'a',
  'new',
  'bag',
  'and',
  'the',
  'shape',
  'of',
  'the',
  'food',
  'is',
  'different',
  'they',
  'tried',
  'the',
  'new',
  'food',
  'when',
  'i',
  'first',
  'put',
  'it',
  'in',
  'their',
  'bowls',
  'and',
  'now',
  'the',
  'bowls',
  'sit',
  'full',
  'and',
  'the',
  'kitties',
  'will',
  'not',
  'touch',
  'the',
  'food',
  "i've",
  'noticed',
  'similar',
  'reviews',
  'related',
  'to',
  'formula',
  'changes',
  'in',
  'the',
  'past',
 

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de rows en el corpus:", w2v_model.corpus_count)

Cantidad de rows en el corpus: 52268


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 15715


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 27256568.0
Loss after epoch 1: 24615960.0
Loss after epoch 2: 17884480.0
Loss after epoch 3: 6954800.0
Loss after epoch 4: 6825232.0
Loss after epoch 5: 6678520.0
Loss after epoch 6: 6513176.0
Loss after epoch 7: 6354504.0
Loss after epoch 8: 6206144.0
Loss after epoch 9: 6049184.0
Loss after epoch 10: 5889280.0
Loss after epoch 11: 5738816.0
Loss after epoch 12: 5580048.0
Loss after epoch 13: 1671016.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0


(68716621, 92894220)

Dado a que se ha tardado, vamos a proceder a guardarlo

In [ ]:
# Save the model to a file
import pickle
with open('/content/drive/MyDrive/Maestria/NLP/w2v_model_1.pkl', 'wb') as f:
    pickle.dump(w2v_model, f)

In [ ]:
# # Load the model from a file
import pickle
with open('/content/drive/MyDrive/Maestria/NLP/w2v_model_1.pkl', 'rb') as f:
    w2v_model = pickle.load(f)

In [ ]:
# prompt: most frequent words on w2v_model

print(w2v_model.wv.index_to_key)


['the', 'i', 'and', 'to', 'a', 'it', 'of', 'this', 'br', 'is', 'in', 'was', 'that', 'not', 'for', 'my', 'but', 'have', 'they', 'with', 'you', 'on', 'like', 'so', 'are', 'product', 'as', 'be', 'from', 'these', 'had', 'taste', 'one', 'at', 'all', 'them', 'food', 'just', 'would', 'if', 'or', 'no', 'very', 'out', 'me', 'were', 'when', 'will', 'we', 'coffee', 'what', 'good', 'has', 'amazon', 'even', 'can', 'more', 'she', 'about', 'buy', 'because', 'there', "don't", 'up', "it's", 'other', 'flavor', 'do', 'get', 'only', 'your', 'after', 'an', 'by', 'some', 'tea', 'really', 'been', 'am', 'now', 'their', 'than', 'which', 'dog', 'again', 'box', 'any', 'her', 'made', 'tried', 'first', 'time', 'much', 'bought', 'bad', 'eat', 'did', 'then', "i'm", 'got', 'too', 'same', 'never', 'money', 'could', 'bag', '2', 'also', 'water', 'know', 'he', 'back', 'try', 'thought', 'order', 'ordered', 'make', 'two', 'better', 'way', '3', '1', 'ingredients', 'cup', 'little', 'over', 'products', 'should', 'price', 'use

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["disappointed"], topn=10)

[('displeased', 0.6394932866096497),
 ('disppointed', 0.6172678470611572),
 ('underwhelmed', 0.5950865149497986),
 ('dissapointed', 0.5787693858146667),
 ('disapointed', 0.5681432485580444),
 ('dissappointed', 0.5511528253555298),
 ('dissatisfied', 0.5364039540290833),
 ('unimpressed', 0.508150577545166),
 ('unhappy', 0.4941144287586212),
 ('unsatisfied', 0.49280908703804016)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["happy"], topn=10)

[('measure', 0.08512692898511887),
 ('flat', 0.056962836533784866),
 ('microwave', 0.04704785719513893),
 ('athletes', 0.046134281903505325),
 ('erucic', 0.04166581854224205),
 ('bottled', 0.037511494010686874),
 ('cosmetics', 0.0366348922252655),
 ('pros', 0.03528863564133644),
 ('element', 0.033686164766550064),
 ('poison', 0.03368280082941055)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('three', 0.4505799114704132),
 ('eighteen', 0.441622793674469),
 ('six', 0.4406546652317047),
 ('delays', 0.4130329191684723),
 ('platic', 0.4019007384777069),
 ('two', 0.3979072570800781),
 ('five', 0.3819637894630432),
 ('4', 0.38148897886276245),
 ('cauldrons', 0.37983864545822144),
 ('5', 0.3743341863155365)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["worst"], topn=5)

[('nastiest', 0.5621068477630615),
 ('foulest', 0.5415406227111816),
 ('grossest', 0.5160967707633972),
 ('saltiest', 0.500125527381897),
 ('driest', 0.49923449754714966)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["stick"])

[('obvious', 0.07993273437023163),
 ('suddenly', 0.07290980964899063),
 ('carcinogen', 0.05927543714642525),
 ('helpful', 0.05730977654457092),
 ('guidelines', 0.05494166538119316),
 ('conducted', 0.05423187464475632),
 ('2010', 0.052661050111055374),
 ('canister', 0.052559662610292435),
 ('unusable', 0.05248876288533211),
 ('treatment', 0.0524781160056591)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_ = w2v_model.wv.get_vector("bad")
print(vector_)

[-7.99017251e-02  1.13713332e-01  2.20451839e-02  1.53673708e-01
 -1.78251609e-01 -1.81794036e-02  3.05200130e-01  1.66440353e-01
 -1.25786334e-01 -1.63489699e-01  3.42027366e-01 -1.30838245e-01
 -1.09361507e-01 -5.77664554e-01  9.20996368e-02  6.96838424e-02
  2.40386307e-01  7.43795782e-02  8.68586674e-02 -1.10857308e-01
 -1.00294150e-01 -1.99301392e-01  2.35869050e-01 -6.47736154e-03
 -1.41481265e-01 -1.40102729e-01 -4.66125548e-01  2.07122773e-01
 -2.45690972e-01 -3.82221699e-01  6.44935787e-01  7.06887320e-02
  1.05916783e-02 -1.72760665e-01  1.41239241e-01 -1.01981618e-01
  1.36399835e-01 -4.58266526e-01 -2.56503150e-02  3.06776136e-01
 -3.37309092e-01  4.34115350e-01 -1.40528195e-02  1.86573401e-01
  4.58138257e-01  2.75206685e-01  8.91981274e-03 -3.87635618e-01
  2.01649666e-01  2.89049029e-01 -2.22361922e-01  1.40870642e-02
  1.94141880e-01 -1.83710381e-01 -3.77238929e-01  2.87746459e-01
 -1.65258616e-01 -3.17709416e-01  7.88928419e-02 -2.80324784e-05
 -1.14826642e-01 -1.27146

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_)

[('bad', 1.0),
 ('bum', 0.4949302077293396),
 ('gaging', 0.47831231355667114),
 ('wierd', 0.4766225516796112),
 ('good', 0.46131280064582825),
 ('briney', 0.4573202431201935),
 ('perfumey', 0.43581148982048035),
 ('helpless', 0.4340755343437195),
 ('plasticky', 0.4301340878009796),
 ('wicked', 0.429080992937088)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["regret"], topn=10)

[('regretted', 0.42980119585990906),
 ('regretting', 0.42263978719711304),
 ('regretful', 0.40428364276885986),
 ('astonished', 0.38491201400756836),
 ('entirelypets', 0.38447830080986023),
 ('regreted', 0.38371434807777405),
 ('upd', 0.37393608689308167),
 ('confess', 0.3720146715641022),
 ('wasted', 0.37132713198661804),
 ('stared', 0.3692549169063568)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

## 5 estrellas

Ahora vamos a repetir para las reseñas positivas (5 estrellas)

In [ ]:
#Me voy a quedar solo con la columna Text y las reviews que son apriori malas
t5=df[df['Score'] == 5]['Text'].to_frame()

In [ ]:
t5.head(30)

,Text
0,I have bought several of the Vitality canned d...
4,Great taffy at a great price. There was a wid...
6,This saltwater taffy had great flavors and was...
7,This taffy is so good. It is very soft and ch...
8,Right now I'm mostly just sprouting this so my...
9,This is a very healthy dog food. Good for thei...
10,I don't know if it's the cactus or the tequila...
11,One of my boys needed to lose some weight and ...
14,The Strawberry Twizzlers are my guilty pleasur...
15,My daughter loves twizzlers and this shipment ...


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for index, row in t5.iterrows():
  sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['i',
  'have',
  'bought',
  'several',
  'of',
  'the',
  'vitality',
  'canned',
  'dog',
  'food',
  'products',
  'and',
  'have',
  'found',
  'them',
  'all',
  'to',
  'be',
  'of',
  'good',
  'quality',
  'the',
  'product',
  'looks',
  'more',
  'like',
  'a',
  'stew',
  'than',
  'a',
  'processed',
  'meat',
  'and',
  'it',
  'smells',
  'better',
  'my',
  'labrador',
  'is',
  'finicky',
  'and',
  'she',
  'appreciates',
  'this',
  'product',
  'better',
  'than',
  'most'],
 ['great',
  'taffy',
  'at',
  'a',
  'great',
  'price',
  'there',
  'was',
  'a',
  'wide',
  'assortment',
  'of',
  'yummy',
  'taffy',
  'delivery',
  'was',
  'very',
  'quick',
  'if',
  'your',
  'a',
  'taffy',
  'lover',
  'this',
  'is',
  'a',
  'deal']]

### 2 - Crear los vectores (word2vec)

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_5 = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model_5.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de rows en el corpus:", w2v_model_5.corpus_count)

Cantidad de rows en el corpus: 363122


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model_5.wv.index_to_key))

Cantidad de words distintas en el corpus: 35795


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_5.train(sentence_tokens,
                 total_examples=w2v_model_5.corpus_count,
                 epochs=4,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 91712232.0
Loss after epoch 1: 41559432.0
Loss after epoch 2: 946064.0
Loss after epoch 3: 0.0


(81423998, 109617980)

Dado a que se ha tardado, vamos a proceder a guardarlo

In [ ]:
# Save the model to a file
import pickle
with open('/content/drive/MyDrive/Maestria/NLP/w2v_model_5.pkl', 'wb') as f:
    pickle.dump(w2v_model_5, f)

In [ ]:
# # Load the model from a file
import pickle
with open('/content/drive/MyDrive/Maestria/NLP/w2v_model_5.pkl', 'rb') as f:
    w2v_model_5 = pickle.load(f)

In [ ]:
# prompt: most frequent words on w2v_model

print(w2v_model_5.wv.index_to_key)


['the', 'i', 'and', 'a', 'to', 'it', 'of', 'is', 'this', 'br', 'for', 'in', 'my', 'that', 'have', 'with', 'you', 'are', 'but', 'they', 'on', 'as', 'not', 'was', 'so', 'these', 'them', 'like', 'great', 'good', 'at', 'or', 'one', 'very', 'just', 'be', 'love', 'all', 'coffee', 'product', 'taste', 'we', 'if', 'tea', 'can', 'from', 'flavor', 'when', 'has', "it's", 'food', 'more', 'will', 'me', 'had', 'amazon', 'than', 'out', 'get', 'no', 'other', 'up', 'best', 'about', 'some', 'really', 'only', 'would', 'also', 'too', 'use', 'time', 'your', 'much', 'an', 'find', 'been', "don't", 'tried', 'little', 'price', 'he', "i've", 'well', 'am', 'now', 'she', 'make', 'because', 'by', 'buy', 'even', 'do', "i'm", 'which', 'try', 'eat', 'our', 'what', 'dog', 'there', 'any', 'better', 'after', 'her', 'were', 'chocolate', 'cup', 'their', '2', 'found', 'free', 'first', 'delicious', 'day', 'sugar', 'bag', 'used', 'water', 'then', 'favorite', 'store', 'sweet', 'recommend', 'drink', 'go', 'loves', '1', 'over', 

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model_5.wv.most_similar(positive=["good"], topn=10)

[('great', 0.6515969634056091),
 ('passable', 0.6435890197753906),
 ('gr8', 0.6434903144836426),
 ('gooooooood', 0.6230233311653137),
 ('goood', 0.6195891499519348),
 ('goooood', 0.6164782047271729),
 ('grat', 0.6030125021934509),
 ('repugnant', 0.600609302520752),
 ('wonderfull', 0.6003708243370056),
 ('gooood', 0.5968116521835327)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model_5.wv.most_similar(negative=["bad"], topn=10)

[('masala', 0.015147429890930653),
 ('garden', 0.008480235934257507),
 ('center', 0.0070176455192267895),
 ('lines', 0.006989947520196438),
 ('edition', 0.0021000762935727835),
 ('art', -0.0015166959492489696),
 ('towards', -0.002995567861944437),
 ('ultra', -0.00552248302847147),
 ('eacute', -0.005833382252603769),
 ('asin', -0.00885782577097416)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model_5.wv.most_similar(positive=["four"], topn=10)

[('three', 0.7048811316490173),
 ('six', 0.7000693678855896),
 ('two', 0.6872736215591431),
 ('five', 0.6518808603286743),
 ('seven', 0.6505892872810364),
 ('4', 0.6351191401481628),
 ('twelve', 0.5944907069206238),
 ('nine', 0.5797582864761353),
 ('eight', 0.5796050429344177),
 ('3', 0.5774251222610474)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model_5.wv.most_similar(positive=["cost"], topn=5)

[('costs', 0.6279570460319519),
 ('price', 0.5882816314697266),
 ('charge', 0.5467729568481445),
 ('markup', 0.5051999688148499),
 ('charges', 0.5048133730888367)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model_5.wv.most_similar(negative=["marvelous"])

[('holistic', 0.04017416015267372),
 ('hospital', 0.029849976301193237),
 ('independent', 0.016371950507164),
 ('insulin', 0.015904080122709274),
 ('enfamil', 0.013624505139887333),
 ('younger', 0.008259242400527),
 ('petsmart', 0.008146792650222778),
 ('vet', 0.007579361088573933),
 ('owner', 0.006659234873950481),
 ('pumping', 0.001378475222736597)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_ = w2v_model_5.wv.get_vector("marvelous")
print(vector_)

[ 0.3531681  -0.12132021  0.00221491  0.22010617 -0.30606657 -0.29387122
 -0.32684913  0.2844981   0.02839503  0.33198306  0.06553318  0.13275173
  0.04347719  0.22340773 -0.34606332 -0.201333    0.05529996 -0.25010622
  0.01324817 -0.49697164  0.09879162 -0.15420502  0.04217108 -0.00309426
  0.117844    0.15033172 -0.5306241   0.13153964 -0.33517236 -0.02727094
  0.0585319  -0.0437362  -0.1129959  -0.2145512  -0.01119625  0.01114088
 -0.08009529  0.05740603  0.13677636  0.01990313 -0.02527263 -0.63276285
  0.13197686  0.00723176  0.16851321  0.15483749 -0.10140974 -0.2926245
  0.64902073  0.07676709 -0.04084165  0.10689243  0.01886271 -0.02655272
  0.14315666 -0.03799014 -0.07856377 -0.08519191 -0.04614546 -0.21554331
  0.2592782   0.3346153  -0.21338193  0.15533116 -0.1970482   0.38939592
 -0.19888914  0.13413747 -0.01651412  0.18518424 -0.29215613  0.30612573
  0.03954681  0.36616057 -0.26567826  0.14313844  0.2752876  -0.1255125
  0.11542774  0.39361477  0.30199143 -0.64773786  0.0

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_5.wv.most_similar(vector_)

[('marvelous', 1.0),
 ('wonderful', 0.6698573231697083),
 ('sublime', 0.5855032205581665),
 ('wonderfull', 0.5706629753112793),
 ('delightful', 0.5653931498527527),
 ('fantastic', 0.552157998085022),
 ('delicious', 0.5506539940834045),
 ('captivating', 0.5490989685058594),
 ('dynamite', 0.5490191578865051),
 ('magnificent', 0.5440505146980286)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model_5.wv.most_similar(positive=["love"], topn=10)

[('loooove', 0.7009639143943787),
 ('loooooove', 0.6622387170791626),
 ('adore', 0.658296525478363),
 ('silki', 0.6088832020759583),
 ('looooove', 0.593902051448822),
 ('looove', 0.5854398012161255),
 ('loove', 0.5799622535705566),
 ("love'em", 0.5789802670478821),
 ('loveeeee', 0.5742121934890747),
 ('afficionados', 0.5718324780464172)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model_5)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model_5,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

Conclusiones:

- Generamos embeddings para las reseñas negativas y positivas de Amazon Fine Food Review. Se compararon para mostrar las diferencias entre los espacios vectoriales.
- Las palabras más similares a "disappointed" en las reseñas negativas son "bad", "terrible", "horrible" y "worst", lo que indica que estos términos están asociados con experiencias negativas con los productos.
- Las palabras más similares a "good" en las reseñas positivas son "excellent", "great", "wonderful" y "amazing", lo que indica que estos términos están asociados con experiencias positivas con los productos.
- La visualización en 2D y 3D de los embeddings muestra que las palabras asociadas con experiencias negativas están agrupadas en una región del espacio vectorial, mientras que las palabras asociadas con experiencias positivas están agrupadas en otra región. Como se pueden ver en la figura [1](https://github.com/perezmillanr/nlp/blob/main/fig%201.png) para el dataset de 1 estrella y [2](https://github.com/perezmillanr/nlp/blob/main/fig%202.png) para el dataset de 5 estrellas.
- Algunos otros ejemplos a destacar son las palabras que son sinónimas, como se puede en la figura [3](https://github.com/perezmillanr/nlp/blob/main/fig%203.png), con "product" y "item". O palabras que representan sentimientos similares, como en la figura [4](https://github.com/perezmillanr/nlp/blob/main/fig%204.png), "taste", "delicius", "sweet", "nice", "flavor".